# Домашнее задание № 10. Машинный перевод

## Задание 1 (6 баллов + 2 доп балла).
Нужно обучить трансформер на этом же или на другом корпусе (можно взять другую языковую пару с того же сайте) и оценивать его на всей тестовой выборке (а не на 10 примерах как сделал я). Используйте метрику BLEU. Найдите лучшие (как минимум 5) переводы согласно этой метрике и проверьте действительно ли они хорошие. Если все переводы нулевые, то пообучайте модель подольше.

Чтобы получить 2 доп балла вам нужно будет придумать как оптимизировать функцию translate. Сейчас она работает только с одним текстом - это не эффективно. Можно генерировать переводы сразу для нескольких текстов (батча). Главная сложность с таким подходом состоит в том, что генерируемые тексты будут заканчиваться в разное время и нужно сделать столько итераций, сколько нужно для завершения всех текстов (т.е. условие на то, что последний токен не равен [END] в текущем коде не сработает).
ВАЖНО - недостаточно просто изменить входной аргумент с text на texts и добавить еще один цикл по texts! Сама модель должна вызываться на нескольких текстах!

In [1]:
# В колабе установите другую версию торча, чтобы семинарская тетрадка работала
!pip install torch==2.0.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 619.9/619.9 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.0/21.0 MB 25.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 849.3/849.3 kB 48.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.8/11.8 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 557.1/557.1 MB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.1/317.1 MB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.6/54.6 MB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.6/102.6 MB 8.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 173.2/173.2 MB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 177.1/177.1 MB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 98.6/98.6 kB 15.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━

In [2]:
!pip install keras -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 11.2 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.1.1 which is incompatible.


In [3]:
import os
os.environ["KERAS_BACKEND"] = "torch"

import keras
import torch
print(keras.__version__)

3.1.1


In [4]:
from tokenizers import BertWordPieceTokenizer

from tokenizers import Tokenizer
from tokenizers.models import WordPiece
from tokenizers.pre_tokenizers import Whitespace
from tokenizers import normalizers
from tokenizers.normalizers import Lowercase
from tokenizers.trainers import WordPieceTrainer
from tokenizers import decoders

import re
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import StratifiedShuffleSplit, train_test_split
from string import punctuation
from collections import Counter
import matplotlib.pyplot as plt
%matplotlib inline

In [5]:
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.ru
!wget https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-test.en

--2024-04-05 13:40:55--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.ru
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 121340806 (116M)
Saving to: ‘opus.en-ru-train.ru’

opus.en-ru-train.ru 100%[===================>] 115.72M  17.0MB/s    in 7.4s    

2024-04-05 13:41:03 (15.7 MB/s) - ‘opus.en-ru-train.ru’ saved [121340806/121340806]

--2024-04-05 13:41:03--  https://data.statmt.org/opus-100-corpus/v1.0/supervised/en-ru/opus.en-ru-train.en
Resolving data.statmt.org (data.statmt.org)... 129.215.32.28
Connecting to data.statmt.org (data.statmt.org)|129.215.32.28|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 67760131 (65M)
Saving to: ‘opus.en-ru-train.en’

opus.en-ru-train.en 100%[===================>]  64.62M  17.3MB/s    in 4.8s    

2024-04-05 13:41:09 (13.5 MB/s) - ‘opus.en-ru

In [6]:
text = open('opus.en-ru-train.ru').read().replace('\xa0', ' ')
f = open('opus.en-ru-train.ru', 'w')
f.write(text)
f.close()
en_sents = open('opus.en-ru-train.en').read().lower().splitlines()
ru_sents = open('opus.en-ru-train.ru').read().lower().splitlines()

In [7]:
en_sents[4], ru_sents[4]

("he'd been out there a few weeks or so.",
 'они тусовались там несколько недель.')

In [8]:
tokenizer_en = Tokenizer(WordPiece(), )
tokenizer_en.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_en.pre_tokenizer = Whitespace()

trainer_en = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]"])
tokenizer_en.train(files=["opus.en-ru-train.en"], trainer=trainer_en )

tokenizer_ru = Tokenizer(WordPiece(), )
tokenizer_ru.normalizer = normalizers.Sequence([Lowercase()])
tokenizer_ru.pre_tokenizer = Whitespace()

trainer_ru = WordPieceTrainer(
          vocab_size=30000, special_tokens=["[UNK]", "[PAD]", "[START]", "[END]", ])
tokenizer_ru.train(files=["opus.en-ru-train.ru"], trainer=trainer_ru )

In [9]:
tokenizer_en.decoder = decoders.WordPiece()
tokenizer_ru.decoder = decoders.WordPiece()

In [10]:
tokenizer_ru.decoder.decode(tokenizer_ru.encode("Я вас категорически приветствую Малолетние дебилы - это малолетние дебилы А на сегодня всё до новых встреч").tokens)

'я вас категорически приветствую малолетние дебилы - это малолетние дебилы а на сегодня всё до новых встреч'

In [11]:
tokenizer_en.save('tokenizer_en')
tokenizer_ru.save('tokenizer_ru')
tokenizer_en = Tokenizer.from_file("tokenizer_en")
tokenizer_ru = Tokenizer.from_file("tokenizer_ru")

In [12]:
def encode(text, tokenizer, target=False):
    if target:
        return [tokenizer.token_to_id('[START]')] + tokenizer.encode(text).ids + \
                [tokenizer.token_to_id('[END]')]
    else:
        return tokenizer.encode(text).ids

In [13]:
X_en = [encode(t, tokenizer_en) for t in en_sents]
X_ru = [encode(t, tokenizer_ru, True) for t in ru_sents]

In [14]:
print('95% eng текстов <= ', np.percentile([len(x) for x in X_en], 95))
#print('99% eng текстов <= ', np.percentile([len(x) for x in X_en], 99))
print('95% rus текстов <= ', np.percentile([len(x) for x in X_ru], 95))
#print('99% rus текстов <= ', np.percentile([len(x) for x in X_ru], 99))

95% eng текстов <=  45.0
95% rus текстов <=  48.0


In [15]:
max_len_en, max_len_ru = 45, 48

In [16]:
PAD_IDX = tokenizer_ru.token_to_id('[PAD]')
PAD_IDX

1

In [17]:
X_en = keras.preprocessing.sequence.pad_sequences(
              X_en, maxlen=max_len_en, padding='post', value=PAD_IDX)

X_ru_out = keras.preprocessing.sequence.pad_sequences(
              [x[1:] for x in X_ru], maxlen=max_len_ru-1, padding='post',
              value=PAD_IDX)

X_ru_dec = keras.preprocessing.sequence.pad_sequences(
              [x[:-1] for x in X_ru], maxlen=max_len_ru-1,
              padding='post', value=PAD_IDX)

In [18]:
en_sents[4], X_en[4]

("he'd been out there a few weeks or so.",
 array([2780,    8,   43, 3010, 2919, 2907,   40, 4357, 5948, 2817, 2853,
          15,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1], dtype=int32))

In [19]:
ru_sents[4], X_ru_out[4], X_ru_dec[4]

('они тусовались там несколько недель.',
 array([2755, 3005, 3008, 6748, 2599, 3298, 4158, 8749,   17,    3,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1], dtype=int32),
 array([   2, 2755, 3005, 3008, 6748, 2599, 3298, 4158, 8749,   17,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1], dtype=int32))

In [20]:
(X_en_train, X_en_valid, X_ru_dec_train, X_ru_dec_valid, X_ru_out_train, X_ru_out_valid) = train_test_split(X_en, X_ru_dec, X_ru_out, test_size=0.05)

In [21]:
def scaled_dot_product_attention(query, key, value, mask):
    # Считаем скалярное произведение между запросом (query) и ключом (key), транспонируя ключ
    matmul_qk = keras.ops.matmul(query, keras.ops.transpose(key, axes=[0, 1, 3, 2]))

    # Получаем глубину (размерность) ключа и преобразуем ее во float
    depth = keras.ops.cast(key.shape[-1], torch.float32)

    # Делим результат скалярного произведения на квадратный корень из глубины
    # Это делается для уменьшения влияния больших значений и стабилизации градиентов во время обучения
    logits = matmul_qk / keras.ops.sqrt(depth)

    # Если есть маска, применяем ее к логитам, чтобы обнулить нежелательные значения
    if mask is not None:
        logits += (mask * -1e9)

    # Применяем функцию softmax для получения весов внимания
    attention_weights = keras.ops.softmax(logits, axis=-1)

    # Умножаем веса внимания на значения (value) для получения итогового результата
    output = keras.ops.matmul(attention_weights, value)

    return output

In [22]:
class MultiHeadAttention(keras.layers.Layer):

    def __init__(self, d_model, num_heads, name="multi_head_attention"):
        super(MultiHeadAttention, self).__init__(name=name)
        self.num_heads = num_heads  # количество голов для внимания
        self.d_model = d_model  # размерность вектора модели

        # Убеждаемся, что размерность модели делится нацело на количество голов
        assert d_model % self.num_heads == 0

        self.depth = d_model // self.num_heads  # размерность каждой головы

        # Создаем полносвязные слои для запроса, ключа и значения
        self.query_dense = keras.layers.Dense(units=d_model)
        self.key_dense = keras.layers.Dense(units=d_model)
        self.value_dense = keras.layers.Dense(units=d_model)

        # Создаем последний полносвязный слой
        self.dense = keras.layers.Dense(units=d_model)

    def split_heads(self, inputs, batch_size):
        # Разделяем входные данные на головы
        inputs = keras.ops.reshape(
            inputs, newshape=(batch_size, -1, self.num_heads, self.depth))
        return keras.ops.transpose(inputs, axes=[0, 2, 1, 3])

    def call(self, inputs):
        query, key, value, mask = inputs['query'], inputs['key'], inputs[
            'value'], inputs.get('mask', None)
        batch_size = query.shape[0]

        # Пропускаем запрос, ключ и значение через соответствующие полносвязные слои
        query = self.query_dense(query)
        key = self.key_dense(key)
        value = self.value_dense(value)

        # Разделяем запрос, ключ и значение на головы
        # то есть просто разрезаем вектора на num_heads частей
        # и сравниваем все части между собой
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # Выполняем механизм внимания с масштабированным скалярным произведением
        scaled_attention = scaled_dot_product_attention(query, key, value, mask)

        scaled_attention = keras.ops.transpose(scaled_attention, axes=[0, 2, 1, 3])

        # Объединяем головы вместе (склеиваем векторы в один)
        concat_attention = keras.ops.reshape(scaled_attention,
                                      (batch_size, -1, self.d_model))

        # Пропускаем объединенное внимание через дополнительный полносвязный слой
        # Он просто добавляет сложности нашей модели
        outputs = self.dense(concat_attention)

        return outputs

In [23]:
def create_padding_mask(x):
    mask = keras.ops.cast(keras.ops.equal(x, PAD_IDX), torch.float32)
    # (batch_size, 1, 1, sequence length)
    return mask[:, None, None, :]

In [24]:
def create_look_ahead_mask(x):
    # нужно создать треугольную маску, с помощью которой мы закроем
    # для каждого токена все последующие токены
    seq_len = x.shape[1]
    ones_mask = keras.ops.ones((1, seq_len, seq_len), dtype="int32")
    row_index = keras.ops.cumsum(ones_mask, axis=-2)
    col_index = keras.ops.cumsum(ones_mask, axis=-1)
    look_ahead_mask = ~ keras.ops.greater_equal(row_index, col_index)
    padding_mask = create_padding_mask(x)
    return keras.ops.maximum(look_ahead_mask, padding_mask)

In [25]:
class PositionalEncoding(keras.layers.Layer):

    def __init__(self, position, d_model):
        super(PositionalEncoding, self).__init__()
        self.pos_encoding = self.positional_encoding(position, d_model)

    def get_angles(self, position, i, d_model):
        angles = 1 / keras.ops.power(10000, (2 * (i // 2)) / d_model)
        return keras.ops.multiply(position, angles)

    def positional_encoding(self, position, d_model):
        angle_rads = self.get_angles(
            position=torch.arange(0, position, dtype=torch.float32)[:, None],
            i=torch.arange(0, d_model, dtype=torch.float32)[None, :],
            d_model=d_model)
        sines = keras.ops.sin(angle_rads[:, 0::2])
        cosines = keras.ops.cos(angle_rads[:, 1::2])

        pos_encoding = keras.ops.concatenate([sines, cosines], axis=-1)
        pos_encoding = pos_encoding[None, ...]
        return keras.ops.cast(pos_encoding, torch.float32)

    def call(self, inputs):
        return inputs + self.pos_encoding[:, :inputs.shape[1], :]

In [26]:
def encoder_layer(units, d_model, num_heads, dropout, name="encoder_layer"):
    inputs = keras.Input(shape=(None, d_model), name="inputs")
    padding_mask = keras.Input(shape=(1, 1, None), name="padding_mask")

    #call_mha
    attention = MultiHeadAttention(
      d_model, num_heads, name="attention")({
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': padding_mask
      })
    attention = keras.layers.Dropout(rate=dropout)(attention)
    attention = keras.layers.LayerNormalization(
      epsilon=1e-6)(inputs + attention)

    outputs = keras.layers.Dense(units=units, activation='relu')(attention)
    outputs = keras.layers.Dense(units=d_model)(outputs)
    outputs = keras.layers.Dropout(rate=dropout)(outputs)
    outputs = keras.layers.LayerNormalization(
      epsilon=1e-6)(attention + outputs)

    return keras.Model(
      inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [27]:
def encoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name="encoder"):
    inputs = keras.Input(shape=(max_len,), name="inputs")
    padding_mask = keras.Input(shape=(1, 1, None), name="padding_mask")

    embeddings = keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= keras.ops.sqrt(keras.ops.cast(d_model, torch.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = encoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name="encoder_layer_{}".format(i),
        )([outputs, padding_mask])

    return keras.Model(inputs=[inputs, padding_mask], outputs=outputs, name=name)

In [28]:
def decoder_layer(units, d_model, num_heads, dropout, name="decoder_layer"):
    inputs = keras.Input(shape=(None, d_model), name="inputs")
    enc_outputs = keras.Input(shape=(None, d_model), name="encoder_outputs")
    look_ahead_mask = keras.Input(
      shape=(1, None, None), name="look_ahead_mask")
    padding_mask = keras.Input(shape=(1, 1, None), name='padding_mask')

    attention1 = MultiHeadAttention(
      d_model, num_heads, name="attention_1")(inputs={
          'query': inputs,
          'key': inputs,
          'value': inputs,
          'mask': look_ahead_mask
      })
    attention1 = keras.layers.LayerNormalization(
      epsilon=1e-6)(attention1 + inputs)

    attention2 = MultiHeadAttention(
      d_model, num_heads, name="attention_2")(inputs={
          'query': attention1,
          'key': enc_outputs,
          'value': enc_outputs,
          'mask': padding_mask
      })
    attention2 = keras.layers.Dropout(rate=dropout)(attention2)
    attention2 = keras.layers.LayerNormalization(
      epsilon=1e-6)(attention2 + attention1)

    outputs = keras.layers.Dense(units=units, activation='relu')(attention2)
    outputs = keras.layers.Dense(units=d_model)(outputs)
    outputs = keras.layers.Dropout(rate=dropout)(outputs)
    outputs = keras.layers.LayerNormalization(
      epsilon=1e-6)(outputs + attention2)

    return keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [29]:
def decoder(vocab_size,
            num_layers,
            units,
            d_model,
            num_heads,
            dropout,
            max_len,
            name='decoder'):
    inputs = keras.Input(shape=(max_len,), name='inputs')
    enc_outputs = keras.Input(shape=(None, d_model), name='encoder_outputs')
    look_ahead_mask = keras.Input(
      shape=(1, None, None), name='look_ahead_mask')
    padding_mask = keras.Input(shape=(1, 1, None), name='padding_mask')

    embeddings = keras.layers.Embedding(vocab_size, d_model)(inputs)
    embeddings *= keras.ops.sqrt(keras.ops.cast(d_model, torch.float32))
    embeddings = PositionalEncoding(max_len, d_model)(embeddings)

    outputs = keras.layers.Dropout(rate=dropout)(embeddings)

    for i in range(num_layers):
        outputs = decoder_layer(
            units=units,
            d_model=d_model,
            num_heads=num_heads,
            dropout=dropout,
            name='decoder_layer_{}'.format(i),
        )(inputs=[outputs, enc_outputs, look_ahead_mask, padding_mask])

    return keras.Model(
      inputs=[inputs, enc_outputs, look_ahead_mask, padding_mask],
      outputs=outputs,
      name=name)

In [30]:
def transformer(vocab_size,
                num_layers,
                units,
                d_model,
                num_heads,
                dropout,
                max_len,
                name="transformer"):
    inputs = keras.Input(shape=(max_len[0],), name="inputs")
    dec_inputs = keras.Input(shape=(max_len[1]-1,), name="dec_inputs")

    enc_padding_mask = keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='enc_padding_mask')(inputs)

    look_ahead_mask = keras.layers.Lambda(
      create_look_ahead_mask,
      output_shape=(1, None, None),
      name='look_ahead_mask')(dec_inputs)

    dec_padding_mask = keras.layers.Lambda(
      create_padding_mask, output_shape=(1, 1, None),
      name='dec_padding_mask')(inputs)

    enc_outputs = encoder(
                          vocab_size=vocab_size[0],
                          num_layers=num_layers,
                          units=units,
                          d_model=d_model,
                          num_heads=num_heads,
                          dropout=dropout,
                          max_len=max_len[0],
                        )(inputs=[inputs, enc_padding_mask])

    dec_outputs = decoder(
                          vocab_size=vocab_size[1],
                          num_layers=num_layers,
                          units=units,
                          d_model=d_model,
                          num_heads=num_heads,
                          dropout=dropout,
                          max_len=max_len[1]-1,
                        )(inputs=[dec_inputs, enc_outputs,
                                  look_ahead_mask, dec_padding_mask])

    outputs = keras.layers.Dense(units=vocab_size[1], name="outputs")(dec_outputs)

    return keras.Model(inputs=[inputs, dec_inputs], outputs=outputs, name=name)

In [31]:
L  = keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none',)

def loss_function(y_true, y_pred):
    loss = L(y_true, y_pred)

    mask = keras.ops.cast(keras.ops.not_equal(y_true, PAD_IDX), torch.float32)
    loss = keras.ops.multiply(loss, mask)

    return keras.ops.mean(loss)

In [32]:
keras.backend.clear_session()

# small model
NUM_LAYERS = 2
D_MODEL = 256
NUM_HEADS = 8

UNITS = 512
DROPOUT = 0.1


model = transformer(
    vocab_size=(tokenizer_en.get_vocab_size(),tokenizer_ru.get_vocab_size()),
    num_layers=NUM_LAYERS,
    units=UNITS,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT,
    max_len=[max_len_en, max_len_ru])


optimizer = keras.optimizers.Adam(
    0.001, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def accuracy(y_true, y_pred):
    return keras.metrics.sparse_categorical_accuracy(y_true, y_pred)


model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])
checkpoint = keras.callbacks.ModelCheckpoint('model_ruen.weights.h5',
                                            monitor='val_loss',
                                            verbose=1,
                                            save_weights_only=True,
                                            save_best_only=True,
                                            mode='min',
                                            save_freq='epoch')

In [33]:
import gc
torch.cuda.empty_cache()
gc.collect()

0

In [34]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [35]:
model.load_weights('/content/drive/MyDrive/Colab Notebooks/model_ruen.weights.h5')

/usr/local/lib/python3.10/dist-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 2 variables whereas the saved optimizer has 178 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


In [ ]:
model.fit((X_en_train, X_ru_dec_train), X_ru_out_train,
             validation_data=((X_en_valid, X_ru_dec_valid), X_ru_out_valid),
             batch_size=200,
             epochs=3,
             callbacks=[checkpoint]
             )

Epoch 1/3
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 0s 338ms/step - accuracy: 0.0904 - loss: 1.6020
Epoch 1: val_loss improved from inf to 0.98785, saving model to model_ruen.weights.h5
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 1651s 348ms/step - accuracy: 0.0904 - loss: 1.6019 - val_accuracy: 0.1471 - val_loss: 0.9878
Epoch 2/3
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 0s 340ms/step - accuracy: 0.1481 - loss: 0.9654
Epoch 2: val_loss improved from 0.98785 to 0.88492, saving model to model_ruen.weights.h5
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 1660s 349ms/step - accuracy: 0.1481 - loss: 0.9654 - val_accuracy: 0.1596 - val_loss: 0.8849
Epoch 3/3
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 0s 339ms/step - accuracy: 0.1593 - loss: 0.8687
Epoch 3: val_loss improved from 0.88492 to 0.84921, saving model to model_ruen.weights.h5
4750/4750 ━━━━━━━━━━━━━━━━━━━━ 1655s 348ms/step - accuracy: 0.1593 - loss: 0.8687 - val_accuracy: 0.1648 - val_loss: 0.8492


In [36]:
@torch.no_grad()
def translate(text):
    input_ids = encode(text, tokenizer_en, target=False)

    input_ids = keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
                                      [input_ids], maxlen=max_len_en, padding='post',
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX), torch.int32)



    output_ids = [tokenizer_ru.token_to_id('[START]') ]

    pred = model((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
                                      [output_ids], maxlen=max_len_ru-1, padding='post',
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX), torch.int32))).cpu().numpy()


    while pred.argmax(2)[0][-1] not in [tokenizer_ru.token_to_id('[END]')]:
        if len(output_ids) >= max_len_ru:
            break
        # можно занизить скор тэга UNK чтобы он никогда не генерировался
        pred[:, :, tokenizer_ru.token_to_id('[UNK]')] = -100

        output_ids.append(pred.argmax(2)[0][-1])
        pred = model((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(
                                      [output_ids], maxlen=max_len_ru-1, padding='post',
                                       # важно не забыть паддинг с нужным id
                                       value=PAD_IDX), torch.int32))).cpu().numpy()

    return tokenizer_ru.decode(output_ids[1:], )

In [37]:
translate("I have consumed a banana today")

'я потреб сегодня бана'

### Функция по переводу сразу нескольких текстов сразу

In [38]:
@torch.no_grad()
def multi_translate(texts):
    input_ids = [encode(text, tokenizer_en, target=False) for text in texts]

    input_ids = keras.ops.cast(keras.preprocessing.sequence.pad_sequences(input_ids, maxlen=max_len_en, padding='post', value=PAD_IDX), torch.int32)
    output_ids = [[tokenizer_ru.token_to_id('[START]') ] for i in range(len(texts))]
    while True:
      pred = model((input_ids, keras.ops.cast(keras.preprocessing.sequence.pad_sequences(output_ids, maxlen=max_len_ru-1, padding='post', value=PAD_IDX), torch.int32))).cpu().numpy()
      quit = True
      for i, p in enumerate(pred):
        if p.argmax(1)[-1] not in [tokenizer_ru.token_to_id('[END]')]:
          quit = False
          p[:, tokenizer_ru.token_to_id('[UNK]')] = -100
          output_ids[i].append(p.argmax(1)[-1])
      if len(output_ids[0]) > max_len_ru or quit:
        break

    return [tokenizer_ru.decode(out_ids[1:],) for out_ids in output_ids]

In [39]:
multi_translate(["He really needs to take a shower",
                 "The bridge got destroyed",
                 "Yes, London. You know: fish, chips, cup 'o tea, bad food, worse weather, Mary fucking Poppins",
                 "According to all known laws of aviation, there is no way a bee should be able to fly"])

['ему нужно принять душ.',
 'мост уничто уничто. мост уничто',
 'да,, вы знаете : рыба,,,,,...... что,,,,,,,, как,,,,,,,,,,,, как...',
 'по всем известным законам вопросамррррррррррррррррррр,,,,, как не,,, не может']

In [40]:
en_sents_test = open('opus.en-ru-test.en').read().lower().splitlines()

### Пытаемся запустить новую функцию


In [43]:
preds = multi_translate(en_sents_test[:100])

KeyboardInterrupt: 

#### Она хочет 10 ГБ оперативки на всю тестовую выборку, и если мы даём её только 100 элементов, она уже начинает обучаться бесконечно. В общем, попробуем более очевидный вариант.

In [44]:
preds_one_by_one = []

for text in en_sents_test:
  translation = translate(text)
  preds_one_by_one.append(translation)

In [46]:
preds_one_by_one[:10]

['если ты оставайся там. только там. не...,,,, ли,,,,,, что,,,,,,,,,,,,, только ты не',
 'я не знаю, как ты,, ты,, все эти.... в каждый день. я не знаю знаю...................',
 'возможно,,, в посредни время естьком..........,,,,,,,,,,, возможно,,,,,,.....',
 'как долго ты хочешь забрать его? его..... как ты,,,,, как он,,,,,,,,,,,, ему....',
 '1 апреля президент наго наго наго наго наго президента республики бак саак саак саак саак саакян встретился начальник генерального штаба сил республики ра ра ра ра ра ра ра ра...........',
 'г - - - - - - - - жа жа также отмечает, - -. он не. отметил. также.',
 'ты не любишь куриный суп??!',
 'posted : apr 2005 :',
 'я думал,, он,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,, я,',
 '« : июнь 19,, 19, :']

In [48]:
en_sents_test[:10]

['if you only stay there.',
 "i don't know how you do it, pop, carrying these boxes around every day.",
 'we might have a slight edge in mediation.',
 'how long is it going to take you to get him what he needs?',
 "on 1 april president of the nagorno karabagh republic bako sahakyan met head of the general staff of the republic of armenia's armed forces colonel-general yuri khachaturov.",
 'mr priesner also noted that the e-justice management system has not only improved case management, but has also led to a significant streamlining in procedures.',
 "you don't like chicken noodle soup?",
 'posted: 14 may 2005, 20:31',
 'now, for a minute, i thought maybe he was being tailed.',
 '« : 26 октябрь 2017, 06:50:24 »']

In [49]:
ru_sents_test = open('opus.en-ru-test.ru').read().lower().splitlines()
ru_sents_test[:10]

['только бы не вылететь.',
 'и как ты только справляешься, папа, таская эти коробки взад-вперед целый день.',
 'возможно, у нас есть небольшое преимущество в переговорах.',
 'сколько времени вы будете делать то, что ему нужно?',
 '1 апреля президент нкр бако саакян принял начальника генштаба вооруженных сил республики армения генерал-полковника юрия хачатурова.',
 'г-н приснер также упомянул, что система электронного правосудия не только позволила улучшить процесс ведения дел, но также способствует значительному упорядочению процедур.',
 '- неплохо, да.',
 'posted: 15 dec 2006, 00:07',
 'и на минутку я подумал, что за ним могут следить.',
 '«: 11 октябрь 2011, 17:15:34»']

### BLEU

In [51]:
import nltk

bleus = []

for i, t in enumerate(preds_one_by_one):
    reference = tokenizer_ru.encode(t).tokens
    hypothesis = tokenizer_ru.encode(ru_sents_test[i]).tokens

    bleus.append(round(nltk.translate.bleu_score.sentence_bleu([reference], hypothesis,  auto_reweigh=True), 3))

In [52]:
(sum(bleus)/len(bleus))*100

0.8989500000000002


## Задание 2 (2 балла).
Прочитайте главу про машинный перевод у Журафски и Маннига - https://web.stanford.edu/~jurafsky/slp3/13.pdf
Ответьте своими словами в чем заключается техника back translation? Для чего она применяется и что позволяет получить? Опишите по шагам как ее применить к паре en->ru на данных из семинара. Сколько моделей понадобится? Сколько запусков обучения нужно будет сделать?

Ответ должен содержать как минимум 10 предложений.

Ответ:
Основная идея back translation заключается в предположении, что хотя параллельные корпуса могут быть ограничены для определенных языков или доменов, мы часто можем найти большой одноязычный корпус, который можно добавить к меньшим параллельным корпусам.

Основная цель применения back translation - улучшить соответствие исходного текста целевому. Для этого используя небольшой параллельный текст (битекст) на исходном и целевом языках.

Шаги применения back translation:

1) Использование битекста для обучения модели первода "в обратном напривлении": из целевого языка в исходный.

2) Использование модели на одноязыном корпусе на целевом языке для перевода его на исходный язык.

3) Добавление синтетического битекста к тренировочным данным. Интетический битекст - это предложения на естественном целевом языке, поставленные в соответсвие с предложениями на исходном языке, обработанными моделью.

